In [2]:
!pip install pulp

In [498]:
import pulp as op
import numpy as np

In [585]:
p = np.array([1,2,3,4]) #precios de las variedades.
w = np.array([400,400,400,400,400,400,400,400,400]) #capacidad de siembra de los bancos.
v = np.array([100,50,75,200]) # cantidades de variedades disponibles para sembrar.
n = v.shape[0] #Cantidad de variedades.
m = w.shape[0] #Cantidad de camas.
L=np.array([[0,0,0,0]]) #Restricciones de NO compatibilidad por variedad.
Q = np.array([0,0,0,0,1,1,1,2,2]) #Distribución de camas a bloques.

In [586]:
concat = lambda t,e : str(t)+str(e)
beds = [concat(xt,xe) for xt,xe in zip(np.repeat(['Bed'],m*n),np.repeat(np.arange(1,m+1),n))]
varieties = [concat(xt,xe) for xt,xe in zip(np.repeat(['_Var'],m*n),np.tile(np.arange(1,n+1),m))]
bedsXvars = [concat(xt,xe) for xt,xe in zip(beds,varieties)]
c=dict(zip(bedsXvars, np.tile(p,m)*np.repeat(w,n)))
A=dict(zip(bedsXvars,np.repeat(w,n)))

In [587]:
prob = op.LpProblem("Plannting_plan",op.LpMaximize)
beds = op.LpVariable.dicts("BedsVar",bedsXvars,lowBound=0,upBound=1,cat=op.LpContinuous)
prob += op.lpSum([c[i]*beds[i] for i in bedsXvars]) # Función de Costo, Precio de la variedad * X

Restricción que representa la capacidad de los bancos por variedad

In [588]:
for j in np.arange(0,n):
    rTemp = np.zeros(m*n)
    for h in np.arange(0,m):
        p1 = np.zeros(j) #Cantidad de ceros adelante segun el indice de la cama
        p2 = [w[h]] #Capacidad del banco
        p3 = np.zeros(n-(j+1)) #
        rTemp[h*n:(h*n)+n]=np.concatenate((p1,p2,p3))
    r = dict(zip(bedsXvars,rTemp))
    prob += op.lpSum([r[i]*beds[i] for i in bedsXvars])<=v[j] #Restricción del área de la siembra.

Restricción que representa que solo pueda sembrarse una variedad por cama

In [589]:
for k in np.arange(0,m): #Se crea un restricción por cama.
    p1 = np.zeros(k*n) #Cantidad de ceros adelante segun el indice de la cama
    p2 = np.ones(n) #Cantidad de unos que representan la restriccion
    p3 = np.zeros(((m*n)-((k*n)+n))) #cantidad de ceros al final
    r = dict(zip(bedsXvars,np.concatenate((p1,p2,p3))))
    prob += op.lpSum([r[i]*beds[i] for i in bedsXvars]) <= 1 #Restricción de máximo una variedad por cama.

In [590]:
q = np.unique(Q)
qCount = [len((Q==xq).nonzero()[0]) for xq in q]
f_zq = 0
for c_qi in np.arange(0,len(q)):
    qi=q[c_qi]
    for li in L:
        p1 = np.zeros(n*f_zq)
        p2 = np.tile(li,qCount[c_qi])
        p3 = np.zeros(n*(len(Q)-qCount[c_qi]-f_zq))
        r = dict(zip(bedsXvars,np.concatenate((p1,p2,p3))))
        prob += op.lpSum([r[i]*beds[i] for i in bedsXvars]) == 0
    f_zq += qCount[c_qi]

In [591]:
prob.solve()
print("Estado:",op.LpStatus[prob.status])
print("Objetivo:",op.value(prob.objective))
print("Vaiables:")
for va in prob.variables():
    if va.varValue>0:
        print(va.name, "=", va.varValue)

Estado: Optimal
Objetivo: 1225.0
Vaiables:
BedsVar_Bed1_Var2 = 0.125
BedsVar_Bed2_Var3 = 0.1875
BedsVar_Bed5_Var1 = 0.25
BedsVar_Bed6_Var4 = 0.5


In [592]:
plantingPlan = np.around((np.array([ x.varValue for x in prob.variables()]).reshape(m,n) * np.repeat(w,n).reshape(m,n)),decimals=0)
plantingPlan

array([[  0.,  50.,   0.,   0.],
       [  0.,   0.,  75.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [100.,   0.,   0.,   0.],
       [  0.,   0.,   0., 200.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.]])

Cantidades de Variedades Pendientes Por Sembrar

In [593]:
v_Planted = np.sum(plantingPlan,axis=0)
v-v_Planted

array([0., 0., 0., 0.])

Aréa disponible despues de la siembra

In [594]:
w_Planted = np.sum(plantingPlan,axis=1)
w-w_Planted

array([350., 325., 400., 400., 300., 200., 400., 400., 400.])

In [595]:
prob

Plannting_plan:
MAXIMIZE
400*BedsVar_Bed1_Var1 + 800*BedsVar_Bed1_Var2 + 1200*BedsVar_Bed1_Var3 + 1600*BedsVar_Bed1_Var4 + 400*BedsVar_Bed2_Var1 + 800*BedsVar_Bed2_Var2 + 1200*BedsVar_Bed2_Var3 + 1600*BedsVar_Bed2_Var4 + 400*BedsVar_Bed3_Var1 + 800*BedsVar_Bed3_Var2 + 1200*BedsVar_Bed3_Var3 + 1600*BedsVar_Bed3_Var4 + 400*BedsVar_Bed4_Var1 + 800*BedsVar_Bed4_Var2 + 1200*BedsVar_Bed4_Var3 + 1600*BedsVar_Bed4_Var4 + 400*BedsVar_Bed5_Var1 + 800*BedsVar_Bed5_Var2 + 1200*BedsVar_Bed5_Var3 + 1600*BedsVar_Bed5_Var4 + 400*BedsVar_Bed6_Var1 + 800*BedsVar_Bed6_Var2 + 1200*BedsVar_Bed6_Var3 + 1600*BedsVar_Bed6_Var4 + 400*BedsVar_Bed7_Var1 + 800*BedsVar_Bed7_Var2 + 1200*BedsVar_Bed7_Var3 + 1600*BedsVar_Bed7_Var4 + 400*BedsVar_Bed8_Var1 + 800*BedsVar_Bed8_Var2 + 1200*BedsVar_Bed8_Var3 + 1600*BedsVar_Bed8_Var4 + 400*BedsVar_Bed9_Var1 + 800*BedsVar_Bed9_Var2 + 1200*BedsVar_Bed9_Var3 + 1600*BedsVar_Bed9_Var4 + 0
SUBJECT TO
_C1: 400 BedsVar_Bed1_Var1 + 400 BedsVar_Bed2_Var1 + 400 BedsVar_Bed3_Var1
 + 40